## 新闻信息提取 开发笔记

* web环境搭建

      基于flask、bootstrap主题搭建web应用

* 服务器安装gunicorn部署web服务
  ```
  # pip install gunicorn

  # gunicorn -w 2 -b 0.0.0.0:8000 app:app

  ```

* 依存分析 

    （优点: 依存分析的可视化和图分析很直观）
  
      工具 ：
         
         --jieba和LTP ，jieba用于分词，LTP用于词性标注和句法分析
      
      LTP提供的模型：
          
          cws.model                分词模型，单文件 
          ner.model                命名实体识别模型，单文件
          pos.model                词性标注模型，单文件
          parser.model             依存分析句法模型，单文件

* NER （named entity reconigation）

In [15]:
import jieba
from collections import Counter


In [16]:
test_str = '''
香港警方于8月14日凌晨3点25分发布声明，对发生在机场的暴力行为予以极严厉的谴责。

声明称，8月13日晚至8月14日凌晨，大量示威者于香港国际机场非法集结，并禁锢、欺凌及殴打一名旅客及一名记者，对他们造成身心伤害，警方对激进示威者的严重暴力行为予以极严厉谴责。

就首宗个案，警方接获香港机场管理局要求，派员到场协助护送救护员及一名受伤旅客安全离开，但警务人员到达后，大批示威者向警务人员不断投掷杂物及照射强力激光，其间一名警务人员被人抢去警棍及被多人袭击，为保护自己以免生命受到威胁或身体受到严重伤害，于是拔枪戒备。

就第二宗个案，一名男记者遭多名示威者粗暴对待、并以索带将其双手捆绑、殴打及禁锢，受害人更一度晕倒。

最后，救护员成功分别将两名受害人送往玛嘉烈医院及北大屿山医院治理。

事件中，警方共拘捕五名男子，涉嫌非法集结、藏有攻击性武器、袭警及破坏社会安宁等。共有两名警务人员受伤，被送往北大屿山医院治理。

案件交由机场警区刑事部跟进调查。

示威者罔顾法纪的暴力行为令人发指，警方予以极严厉谴责。警方强调，会积极对案件作出深入调查，不会放过任何线索，务必将施袭者绳之于法。
'''

In [17]:
jieba.add_word('港独')  #手动添加处理词汇
jieba.add_word('北大屿山')
words = list(jieba.cut(test_str))

In [18]:
print(words)

['\n', '香港', '警方', '于', '8', '月', '14', '日', '凌晨', '3', '点', '25', '分', '发布', '声明', '，', '对', '发生', '在', '机场', '的', '暴力行为', '予以', '极', '严厉', '的', '谴责', '。', '\n', '\n', '声明', '称', '，', '8', '月', '13', '日晚', '至', '8', '月', '14', '日', '凌晨', '，', '大量', '示威者', '于', '香港', '国际', '机场', '非法', '集结', '，', '并', '禁锢', '、', '欺凌', '及', '殴打', '一名', '旅客', '及', '一名', '记者', '，', '对', '他们', '造成', '身心', '伤害', '，', '警方', '对', '激进', '示威者', '的', '严重', '暴力行为', '予以', '极', '严厉', '谴责', '。', '\n', '\n', '就', '首宗', '个案', '，', '警方', '接获', '香港机场', '管理局', '要求', '，', '派员', '到场', '协助', '护送', '救护', '员及', '一名', '受伤', '旅客', '安全', '离开', '，', '但', '警务人员', '到达', '后', '，', '大批', '示威者', '向', '警务人员', '不断', '投掷', '杂物', '及', '照射', '强力', '激光', '，', '其间', '一名', '警务人员', '被', '人', '抢', '去', '警棍', '及', '被', '多', '人', '袭击', '，', '为', '保护', '自己', '以免', '生命', '受到', '威胁', '或', '身体', '受到', '严重', '伤害', '，', '于是', '拔枪', '戒备', '。', '\n', '\n', '就', '第二宗', '个案', '，', '一名', '男', '记者', '遭', '多名', '示威者', '粗暴', '对待', '、', '并', '以索', '带', '将', '其',

In [ ]:

# 词性标注

postagger = Postagger()
postagger.load('/Users/yuxijian/ltp_model/pos.model')
postags = postagger.postag(words)

# 依存句法分析


parser = Parser()
parser.load('/Users/yuxijian/ltp_model/parser.model')
arcs = parser.parse(words, postags)

rely_id = [arc.head for arc in arcs]
relation = [arc.relation for arc in arcs]
print('rely_id', rely_id)
print('relation', relation)

heads = ['Root' if id == 0 else words[id-1] for id in rely_id]
print('heads', heads)

for i in range(len(words)):
    print(relation[i] + '(' + words[i] + ',' + heads[i] + ')')


# # 可视化
#
# from graphviz import Digraph
#
# g = Digraph('测试图片')
# g.node(name='Root')
#
# for word in words:
#     g.node(name=word)
#
# for i in range(len(words)):
#     if relation[i] not in ['HED']:
#         g.edge(words[i], heads[i], label=relation[i])
#     else:
#         if heads[i] == 'Root':
#             g.edge(words[i], 'Root', label=relation[i])
#         else:
#             g.edge(heads[i], 'Root', label=relation[i])
#
# g.view()


# 利用networkx绘制句法分析结果
import networkx as nx
import matplotlib.pyplot as plt
from pylab import mpl

mpl.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 指定默认字体

G = nx.Graph()  # 建立无向图G

# 添加节点
for word in words:
    G.add_node(word)

G.add_node('Root')

# 添加边
for i in range(len(words)):
    G.add_edge(words[i], heads[i])

source = '警方'
target1 = '示威者'
distance1 = nx.shortest_path_length(G, source=source, target=target1)
print("'%s'与'%s'在依存句法分析图中的最短距离为:  %s" % (source, target1, distance1))

# target2 = '受害人'
# distance2 = nx.shortest_path_length(G, source=source, target=target2)
# print("'%s'与'%s'在依存句法分析图中的最短距离为:  %s" % (source, target2, distance2))

nx.draw(G, with_labels=True)
plt.show()
# plt.savefig("undirected_graph.png")

3、